In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fiona
import collections
import matplotlib.pyplot as plt
import pyproj
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio import features
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, timedelta
import geopandas
import os
from shapely import geometry
import numpy as np
import utilities
import pandas as pd
import shutil
import glob
pd.set_option('display.max_columns', None)
import time
import zipfile
import geopandas as gpd
import shapely
import geopandas as gpd
from shapely.geometry import MultiPoint, Polygon
from PIL import Image

In [3]:
path = "bulk/"
dest = "bulk_crops/"
counties = True
parishes = False
cutoff_name = 'Mação'

In [4]:
if parishes:
    level = 3
if counties:
    level=2
    
all_shapes = gpd.read_file(f"gadm36_PRT_shp/gadm36_PRT_{level}.shp")
cutoff_shp = all_shapes[all_shapes[f'NAME_{level}']==cutoff_name].iloc[0].geometry
all_shapes_ptcrs = all_shapes.to_crs(epsg=32629)
cutoff_shp_ptcrs = all_shapes_ptcrs[all_shapes_ptcrs[f'NAME_{level}']==cutoff_name].iloc[0].geometry

In [5]:
outer_square = utilities.polygon_outer_square(cutoff_shp)

In [ ]:
os.makedirs(path, exist_ok=True)
os.makedirs(dest, exist_ok=True)

#get products list from this day
api = SentinelAPI('fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
products = api.query(outer_square,
                     date=(date.today() - timedelta(7), date.today()),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))
products_df = api.to_dataframe(products)

if products_df.empty:
    print('No images found')
else:    
    downloaded_product = utilities.download_most_recent_product(products_df, polygon_to_overlap=cutoff_shp, path=path)

8 available products
Trying to download 2133903c-e90a-4735-8076-e66725241990


Downloading:  28%|██▊       | 339M/1.21G [12:57<31:51, 457kB/s]   

In [ ]:
# copy jp2 to dest
utilities.subfolders_copy(os.path.join(path, downloaded_product['title'] + '.SAFE'), dest)

In [ ]:
# save cropped jp2 with same name 
for i in [img for img in utilities.listdir_nohidden(dest) if img.endswith('.jp2')]:
    for namepart in i.split('/')[-1].split('_'):
        if namepart.startswith('202'):
            imgname = f'{namepart[:4]}-{namepart[4:6]}-{namepart[6:8]}_{namepart[9:11]}:{namepart[11:13]}'
            break
    with rasterio.open(os.path.join(dest, i)) as src:
        out_image, out_transform = rasterio.mask.mask(src, [cutoff_shp_ptcrs], crop=True, nodata=0, all_touched=True)
        out_meta = src.meta.copy() 
    
    out_image = utilities.append_transparency_band(out_image)
            
    with rasterio.open(os.path.join(dest, imgname)+'.png','w',
                       driver='PNG',
                       height=out_image.shape[1],
                       width=out_image.shape[2],
                       dtype=rasterio.uint8,
                       count=out_image.shape[0],
                       compress='lzw') as dst:
        dst.write(np.array(out_image, dtype='uint8'))
    
for f in utilities.listdir_nohidden(dest):
    if '.jp2' in f and not f.startswith('.'):
        os.remove(os.path.join(dest, f))

In [ ]:
shutil.rmtree(os.path.join(path, f'{downloaded_product["title"]}.SAFE'))